This example notebook demonstrates how to write VAME results to an NWB file, using the [ndx-vame]() extension.

It will also include:
- [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) metadata
- [PoseEstimation](https://github.com/rly/ndx-pose) data

Let's start by importing the necessary modules:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import numpy as np
from pynwb import NWBFile, NWBHDF5IO
from pynwb.file import Subject
from ndx_pose import PoseEstimationSeries, PoseEstimation
from ndx_vame import VAMEGroup, MotifSeries, CommunitySeries

In [3]:
# initialize an NWBFile object
nwbfile = NWBFile(
    session_description="session_description",
    identifier="identifier",
    session_start_time=datetime.datetime.now(datetime.timezone.utc),
)

# add a subject to the NWB file
subject = Subject(subject_id="subject1", species="Mus musculus")
nwbfile.subject = subject

# create a device for the camera
camera1 = nwbfile.create_device(
    name="camera1",
    description="camera for recording behavior",
    manufacturer="manufacturer",
)

# a PoseEstimationSeries represents the estimated position of a single marker.
# in this example, we have three PoseEstimationSeries: one for the body and one for each front paw.
data = np.random.rand(100, 2)  # num_frames x (x, y) but can be (x, y, z)
confidence = np.random.rand(100)  # a confidence value for every frame
reference_frame = "(0,0,0) corresponds to ..."
front_left_paw = PoseEstimationSeries(
    name="front_left_paw",
    description="Marker placed around fingers of front left paw.",
    data=data,
    unit="pixels",
    rate=10.,
    reference_frame=reference_frame,
    confidence=confidence,
)

data = np.random.rand(100, 2)  # num_frames x (x, y) but can be (x, y, z)
confidence = np.random.rand(100)  # a confidence value for every frame
body = PoseEstimationSeries(
    name="body",
    description="Marker placed on center of body.",
    data=data,
    unit="pixels",
    rate=10.,
    reference_frame=reference_frame,
    confidence=confidence,
)

data = np.random.rand(100, 2)  # num_frames x (x, y) but can be num_frames x (x, y, z)
confidence = np.random.rand(100)  # a confidence value for every frame
front_right_paw = PoseEstimationSeries(
    name="front_right_paw",
    description="Marker placed around fingers of front right paw.",
    data=data,
    unit="pixels",
    rate=10.,
    reference_frame=reference_frame,
    confidence=confidence,
)

# create a PoseEstimation object
pose_estimation = PoseEstimation(
    name="PoseEstimation",
    pose_estimation_series=[front_left_paw, body, front_right_paw],
    description="Estimated positions of front paws of subject1 using DeepLabCut.",
    original_videos=["path/to/camera1.mp4"],
    dimensions=np.array([[640, 480]], dtype="uint16"),  # pixel dimensions of the video
    devices=[camera1],
    scorer="DLC_resnet50_openfieldOct30shuffle1_1600",
    source_software="DeepLabCut",
    source_software_version="2.3.8",
)

# create a "behavior" processing module to store the PoseEstimation and Skeletons objects
behavior_pm = nwbfile.create_processing_module(
    name="behavior",
    description="processed behavioral data",
)
behavior_pm.add(pose_estimation)

# VAME data
data_motifs = np.random.rand(100, 15)  # (n_samples, n_motifs)
data_communities = np.random.rand(100, 3)  # (n_samples, n_communities)

motif_series = MotifSeries(
    name="MotifSeries",
    data=data_motifs,
    rate=10.,
)
community_series = CommunitySeries(
    name="CommunitySeries",
    data=data_communities,
    rate=10.,
    motif_series=motif_series,
)

vame_group = VAMEGroup(
    name="VAMEGroup",
    motif_series=motif_series,
    community_series=community_series,
    vame_settings="dict containing config",
    pose_estimation=pose_estimation,
)

behavior_pm.add(vame_group)

VAMEGroup abc.VAMEGroup at 0x140553105060368
Fields:
  community_series: CommunitySeries abc.CommunitySeries at 0x140553107402640
Fields:
  comments: no comments
  conversion: 1.0
  data: [[0.43998806 0.53170212 0.66043415]
 [0.75721435 0.10636905 0.11300232]
 [0.09077446 0.90104402 0.34651507]
 [0.62515967 0.17779079 0.7596648 ]
 [0.91372801 0.99420468 0.59863262]
 [0.86930267 0.83095786 0.2281408 ]
 [0.74265926 0.22465448 0.89960072]
 [0.66454641 0.46558438 0.28695362]
 [0.8353196  0.18093138 0.2411628 ]
 [0.71246404 0.14780792 0.89039325]
 [0.19592592 0.88267309 0.39139471]
 [0.93949947 0.97692976 0.28158812]
 [0.95984246 0.44190338 0.54714194]
 [0.63007312 0.81592894 0.41756854]
 [0.25608031 0.01024142 0.91196765]
 [0.54558918 0.39938593 0.8642789 ]
 [0.18729383 0.13108759 0.77533922]
 [0.69892879 0.74304113 0.56961341]
 [0.70005504 0.71939267 0.59555801]
 [0.31418014 0.39885414 0.77687772]
 [0.27215461 0.28960296 0.92367924]
 [0.41287496 0.29738793 0.16972983]
 [0.73899105 0.26289448 0.76332186]
 [0.04071338 0.61557764 0.70857751]
 [0.68167495 0.71634715 0.82019434]
 [0.27341473 0.73677139 0.33551098]
 [0.01010127 0.91466858 0.87697741]
 [0.76134159 0.52011052 0.95520642]
 [0.42175612 0.91265373 0.32957799]
 [0.75514834 0.44401934 0.6026244 ]
 [0.07507509 0.28569872 0.21151653]
 [0.80242932 0.69675729 0.15769747]
 [0.56346139 0.60819426 0.0212604 ]
 [0.33254317 0.42539633 0.90909181]
 [0.50767501 0.06351399 0.86430952]
 [0.35176802 0.44208233 0.95891596]
 [0.5344714  0.61339322 0.8601746 ]
 [0.17266637 0.1760966  0.74465095]
 [0.26197483 0.55994107 0.13549502]
 [0.33726788 0.66497732 0.59620111]
 [0.51476562 0.14076866 0.00335049]
 [0.7960859  0.90641082 0.00488794]
 [0.31334125 0.56487202 0.226219  ]
 [0.81008117 0.39921372 0.83467877]
 [0.18382521 0.75638114 0.42698858]
 [0.73407943 0.63712799 0.08507158]
 [0.84372047 0.02104686 0.23635313]
 [0.06621806 0.60900064 0.33918009]
 [0.0857677  0.41445008 0.10185831]
 [0.88111584 0.44545243 0.88281855]
 [0.80774039 0.34397866 0.75093791]
 [0.95955867 0.05095818 0.66969786]
 [0.63733952 0.58764687 0.83946446]
 [0.36104783 0.76858308 0.69947213]
 [0.31937254 0.1686174  0.34566849]
 [0.58825577 0.21273915 0.10748281]
 [0.32102787 0.87793607 0.03985022]
 [0.72371427 0.92715472 0.17402705]
 [0.70352679 0.66330138 0.90862895]
 [0.49794876 0.16056606 0.83890448]
 [0.29273593 0.79159447 0.04185646]
 [0.22544977 0.96698887 0.77307588]
 [0.8230655  0.14622747 0.79932183]
 [0.11868323 0.81869011 0.80086148]
 [0.46378689 0.99250777 0.89091225]
 [0.7204117  0.3075677  0.74339047]
 [0.95836956 0.19946639 0.49544005]
 [0.84487794 0.37179784 0.45263552]
 [0.66823946 0.44335485 0.41475522]
 [0.39476859 0.66638461 0.43949592]
 [0.18985889 0.39272395 0.53389903]
 [0.85243214 0.30226465 0.00762267]
 [0.09895742 0.54915173 0.34065714]
 [0.73000793 0.9219293  0.21140898]
 [0.66695104 0.18118635 0.80615678]
 [0.58869477 0.63196128 0.3318188 ]
 [0.31026072 0.74770009 0.25482179]
 [0.86574701 0.32017136 0.82023157]
 [0.36996168 0.22324185 0.14610425]
 [0.14012199 0.03810094 0.57865856]
 [0.45516287 0.32010906 0.73730203]
 [0.32644794 0.45984011 0.6269077 ]
 [0.63433348 0.59290693 0.20854109]
 [0.90233648 0.77473339 0.639804  ]
 [0.22720437 0.58038642 0.71718475]
 [0.58403782 0.33364533 0.59914486]
 [0.65448278 0.43950061 0.85980971]
 [0.32968912 0.82905134 0.08803322]
 [0.1139342  0.47366147 0.45689916]
 [0.88695635 0.52025666 0.67793578]
 [0.95853009 0.01440449 0.61858106]
 [0.6387198  0.79381927 0.32477174]
 [0.33081784 0.05783369 0.75891636]
 [0.51084636 0.14769843 0.87483099]
 [0.14026398 0.17013078 0.60685809]
 [0.1815903  0.99052912 0.51745449]
 [0.56344258 0.71265964 0.17635368]
 [0.30811171 0.42507581 0.0127794 ]
 [0.65239602 0.85442191 0.88706445]
 [0.12700319 0.72406958 0.70163793]]
  description: no description
  motif_series: MotifSeries abc.MotifSeries at 0x140553102999504
Fields:
  comments: no comments
  conversion: 1.0
  data: [[0.32210193 0.01265474 0.6616303  ... 0.00657195 0.792935

In [4]:
# write the NWBFile to disk
path = "test_pose.nwb"
with NWBHDF5IO(path, mode="w") as io:
    io.write(nwbfile)

# read the NWBFile from disk and print out the PoseEstimation and Skeleton objects
# as well as the first training frame
with NWBHDF5IO(path, mode="r") as io:
    read_nwbfile = io.read()
    print(read_nwbfile.processing["behavior"]["VAMEGroup"])

VAMEGroup abc.VAMEGroup at 0x140553103149968
Fields:
  community_series: CommunitySeries abc.CommunitySeries at 0x140553104163536
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (100, 3), type "<f8">
  description: no description
  motif_series: MotifSeries abc.MotifSeries at 0x140553102707280
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (100, 15), type "<f8">
  description: no description
  offset: 0.0
  rate: 10.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: n/a

  offset: 0.0
  rate: 10.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: n/a

  motif_series: MotifSeries abc.MotifSeries at 0x140553102707280
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (100, 15), type "<f8">
  description: no description
  offset: 0.0
  rate: 10.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: